In [ ]:
import networkx as nx
import pyomo.core as pyo
import classiq

In [ ]:
from pyomo.environ import *

class FlightReassignmentOptimizer:
    def __init__(self, passengers, flights, delay_matrix, capacity, eligibility_matrix):
        """
        Initialize the optimization model.

        :param passengers: List of passenger IDs
        :param flights: List of flight IDs
        :param delay_matrix: Dictionary {(i, j): delay} for passenger i and flight j
        :param capacity: Dictionary {j: capacity} for each flight j
        :param eligibility_matrix: Dictionary {(i, j): eligibility (0 or 1)} for passenger i and flight j
        """
        self.passengers = passengers
        self.flights = flights
        self.delay_matrix = delay_matrix
        self.capacity = capacity
        self.eligibility_matrix = eligibility_matrix
        
        self.model = ConcreteModel()
        self._build_model()
    
    def _build_model(self):
        # Define sets
        self.model.PASSENGERS = Set(initialize=self.passengers)
        self.model.FLIGHTS = Set(initialize=self.flights)
        
        # Define binary decision variables
        self.model.x = Var(self.model.PASSENGERS, self.model.FLIGHTS, domain=Binary)
        
        # Objective: Minimize total delay
        def objective_rule(model):
            return sum(model.x[i, j] * self.delay_matrix[i, j] 
                       for i in model.PASSENGERS 
                       for j in model.FLIGHTS)
        self.model.objective = Objective(rule=objective_rule, sense=minimize)
        
        # Constraint: Each passenger is assigned to at most one flight
        def assignment_constraint_rule(model, i):
            return sum(model.x[i, j] for j in model.FLIGHTS) <= 1
        self.model.assignment_constraint = Constraint(self.model.PASSENGERS, rule=assignment_constraint_rule)
        
        # # Constraint: Flight capacity
        # def capacity_constraint_rule(model, j):
        #     return sum(model.x[i, j] for i in model.PASSENGERS) <= self.capacity[j]
        # self.model.capacity_constraint = Constraint(self.model.FLIGHTS, rule=capacity_constraint_rule)
        
        # # Constraint: Eligibility
        # def eligibility_constraint_rule(model, i, j):
        #     return model.x[i, j] <= self.eligibility_matrix[i, j]
        # self.model.eligibility_constraint = Constraint(self.model.PASSENGERS, self.model.FLIGHTS, rule=eligibility_constraint_rule)
    
    def get_model(self):
        """
        Return the ConcreteModel object.
        """
        return self.model
    
    def solve(self):
        # Solve the model using a solver
        solver = SolverFactory('glpk')  # Use 'glpk', 'cbc', or 'gurobi' depending on availability
        result = solver.solve(self.model)
        return result
    
    def get_results(self):
        # Extract the results: Assigned passengers and flights
        assignments = []
        for i in self.passengers:
            for j in self.flights:
                if self.model.x[i, j].value == 1:
                    assignments.append((i, j))
        return assignments


In [ ]:
# Example data
passengers = list(range(2))  # 10 passengers indexed 1-10
flights = list(range(7))  # 10 flights indexed 1-10

# Delay matrix: {(passenger, flight): delay_in_minutes}
delay_matrix = {(i, j): abs(j - i) * 10 + 10 for i in passengers for j in flights}  # Example delays

# Flight capacity: {flight: capacity}
capacity = {j: 2 for j in flights}  # Each flight has 2 available seats

# Eligibility matrix: {(passenger, flight): eligibility (0 or 1)}
eligibility_matrix = {(i, j): 1 if (i + j) % 3 != 0 else 0 for i in passengers for j in flights}  # Example eligibility

# Create the optimizer
optimizer = FlightReassignmentOptimizer(passengers, flights, delay_matrix, capacity, eligibility_matrix)

problem_test = optimizer.get_model()

In [ ]:
from classiq.applications.combinatorial_optimization import (
    QAOAConfig,
    get_optimization_solution_from_pyo,
    OptimizerConfig,
)
from classiq.execution import OptimizerType
from classiq import construct_combinatorial_optimization_model, synthesize, show, execute
from classiq.execution import ExecutionPreferences
from classiq.execution import ClassiqBackendPreferences
import pandas as pd
from classiq import *


qaoa_config = QAOAConfig(num_layers=4)
optimizer_config = OptimizerConfig(
    opt_type=OptimizerType.COBYLA, max_iteration=1000
)
model_test = construct_combinatorial_optimization_model(
    pyo_model=problem_test, qaoa_config=qaoa_config
)
qmod_prefs = set_execution_preferences(
    model_test,
    ExecutionPreferences(num_shots=10000, job_name="Flight Reassignment"),
)

mis_quantum_program = synthesize(model_test)
show(mis_quantum_program)

In [ ]:
res = execute(mis_quantum_program).result()

In [ ]:
vqe_result = res[0].value
solution = get_optimization_solution_from_pyo(
    problem_test, vqe_result=vqe_result, penalty_energy=qaoa_config.penalty_energy
)
optimization_result = pd.DataFrame.from_records(solution)
optimization_result.sort_values(by="cost", ascending=True).head(15)